In [38]:
import tensorflow as tf
import matplotlib.pyplot as plt

import collections
import random
import numpy as np
import os
import time
import json
from PIL import Image
import model
import pickle 
import utils 
import argparse

In [39]:
embedding_dim = 256
units = 512
vocab_size = 5001
max_length =10 
attention_features_shape = 64
units = 512

# First, prepare model 
# define image feature extraction  model 
image_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output
image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

# define encoder and decoder model 
encoder = model.CNN_Encoder(embedding_dim)
decoder = model.RNN_Decoder(embedding_dim, units, vocab_size)
encoder.load_weights("/home-3/kjo3@jhu.edu/checkpoints/saved/encoder")
decoder.load_weights("/home-3/kjo3@jhu.edu/checkpoints/saved/decoder")

2021-09-17 19:49:30.971163: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 540.0KiB (rounded to 552960)requested by op Add
Current allocation summary follows.
2021-09-17 19:49:30.971242: I tensorflow/core/common_runtime/bfc_allocator.cc:972] BFCAllocator dump for GPU_0_bfc
2021-09-17 19:49:30.971257: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (256): 	Total Chunks: 315, Chunks in use: 315. 78.8KiB allocated for chunks. 78.8KiB in use in bin. 52.8KiB client-requested in use in bin.
2021-09-17 19:49:30.971267: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (512): 	Total Chunks: 667, Chunks in use: 667. 456.0KiB allocated for chunks. 456.0KiB in use in bin. 421.5KiB client-requested in use in bin.
2021-09-17 19:49:30.971276: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (1024): 	Total Chunks: 156, Chunks in use: 156. 227.8KiB allocated for chunks. 227.8KiB in use in bin. 222.5KiB cli

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,80,192] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add]

ocator.cc:1036] 1 Chunks of size 237568 totalling 232.0KiB
2021-09-17 19:49:30.984038: I tensorflow/core/common_runtime/bfc_allocator.cc:1036] 5 Chunks of size 307200 totalling 1.46MiB
2021-09-17 19:49:30.984044: I tensorflow/core/common_runtime/bfc_allocator.cc:1036] 10 Chunks of size 331776 totalling 3.16MiB
2021-09-17 19:49:30.984050: I tensorflow/core/common_runtime/bfc_allocator.cc:1036] 1 Chunks of size 352256 totalling 344.0KiB
2021-09-17 19:49:30.984056: I tensorflow/core/common_runtime/bfc_allocator.cc:1036] 2 Chunks of size 356352 totalling 696.0KiB
2021-09-17 19:49:30.984062: I tensorflow/core/common_runtime/bfc_allocator.cc:1036] 1 Chunks of size 368640 totalling 360.0KiB
2021-09-17 19:49:30.984068: I tensorflow/core/common_runtime/bfc_allocator.cc:1036] 5 Chunks of size 393216 totalling 1.88MiB
2021-09-17 19:49:30.984074: I tensorflow/core/common_runtime/bfc_allocator.cc:1036] 1 Chunks of size 442368 totalling 432.0KiB
2021-09-17 19:49:30.984081: I tensorflow/core/common_r

In [ ]:
# load tokenizer 
with open("/home-3/kjo3@jhu.edu/tokenizer.pickle", 'rb') as handle:
    tokenizer = pickle.load(handle)


In [ ]:
image = "/home-3/kjo3@jhu.edu/kyuhee/coco/images/val2014/COCO_val2014_000000465545.jpg"
temp_input = tf.expand_dims(utils.load_image(image)[0], 0)
img_tensor_val = image_features_extract_model(temp_input)
img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

In [ ]:
features = encoder(img_tensor_val)

dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
result = []

In [ ]:
for i in range(max_length):
    predictions, hidden, attention_weights = decoder(dec_input,
                                                        features,
                                                        tf.zeros((1, units)))

    predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
    result.append(tokenizer.index_word[predicted_id])

    if tokenizer.index_word[predicted_id] == '<end>':
        print('Prediction Caption:', ' '.join(result))

    dec_input = tf.expand_dims([predicted_id], 0)

In [ ]:
result